In [2]:
import numpy as np
import math
import random

# =============================
#    Toy Protein-Ligand Model
# =============================

POCKET = np.array([
    [0, 0, 0],
    [2, 1, -1],
    [-1, -2, 1],
    [1, -1, 2]
])

NUM_LIGAND_ATOMS = 4
LOWER_BOUND = -5
UPPER_BOUND = 5


# ==========================================
#   Docking Scoring Function (Toy Version)
# ==========================================
def docking_score(ligand):
    score = 0.0

    for l_atom in ligand:
        distances = np.linalg.norm(POCKET - l_atom, axis=1)

        close_contacts = np.sum(np.exp(-distances))         # reward closeness
        clash_penalty = np.sum(np.where(distances < 0.6, 10, 0))  # clash penalty

        score += -close_contacts + clash_penalty

    return score


# ==========================
#   Lévy Flight Function
# ==========================
def levy_flight(Lambda):
    sigma = (math.gamma(1 + Lambda) *
             math.sin(math.pi * Lambda / 2) /
             (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)

    u = np.random.normal(0, sigma, 1)[0]
    v = np.random.normal(0, 1, 1)[0]

    step = u / (abs(v) ** (1 / Lambda))
    return step


# ==========================
#   Cuckoo Search Algorithm
# ==========================
def cuckoo_search(num_nests=20, pa=0.25, max_iter=100):
    dimension = NUM_LIGAND_ATOMS * 3

    nests = np.random.uniform(LOWER_BOUND, UPPER_BOUND, (num_nests, dimension))
    fitness = np.array([docking_score(n.reshape(NUM_LIGAND_ATOMS, 3)) for n in nests])

    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx].copy()
    best_score = fitness[best_idx]

    for it in range(max_iter):

        # Lévy flight from global best
        cuckoo = best_nest + levy_flight(1.5)
        cuckoo = np.clip(cuckoo, LOWER_BOUND, UPPER_BOUND)

        cuckoo_score = docking_score(cuckoo.reshape(NUM_LIGAND_ATOMS, 3))

        # Replace random nest if better
        j = np.random.randint(0, num_nests)
        if cuckoo_score < fitness[j]:
            nests[j] = cuckoo
            fitness[j] = cuckoo_score

        # Abandon some nests
        abandon_mask = np.random.rand(num_nests) < pa
        new_random = np.random.uniform(LOWER_BOUND, UPPER_BOUND, (num_nests, dimension))
        nests[abandon_mask] = new_random[abandon_mask]

        # Re-evaluate fitness
        fitness = np.array([docking_score(n.reshape(NUM_LIGAND_ATOMS, 3)) for n in nests])

        # Update global best
        best_idx = np.argmin(fitness)
        if fitness[best_idx] < best_score:
            best_score = fitness[best_idx]
            best_nest = nests[best_idx].copy()

        # Print progress
        if it % 10 == 0 or it == max_iter - 1:
            print(f"Iteration {it}: Best Score = {best_score:.4f}")

    return best_score, best_nest.reshape(NUM_LIGAND_ATOMS, 3)


# ========================
#          MAIN
# ========================
best_energy, best_ligand = cuckoo_search()

print("\n====== BEST DOCKING POSE FOUND ======")
print("Energy (lower = better):", best_energy)
print("Ligand atom coordinates:")
print(best_ligand)


Iteration 0: Best Score = -1.1223
Iteration 10: Best Score = -1.3505
Iteration 20: Best Score = -1.3553
Iteration 30: Best Score = -1.3601
Iteration 40: Best Score = -1.3601
Iteration 50: Best Score = -1.3601
Iteration 60: Best Score = -1.3601
Iteration 70: Best Score = -1.3601
Iteration 80: Best Score = -1.3601
Iteration 90: Best Score = -1.3601
Iteration 99: Best Score = -1.3601

====== BEST DOCKING POSE FOUND ======
Energy (lower = better): -1.3601388947496216
Ligand atom coordinates:
[[ 1.69021517  0.74481628 -0.51679895]
 [ 0.49855102 -2.20500789  0.80706998]
 [-1.86137921 -2.21752201  4.38605114]
 [ 3.66238965 -1.35373699  3.32897792]]
